In [1]:
import tensorflow as tf
import numpy as np

### 定义TextRNN结构，使用双向的LSTM

In [2]:
class TextRNN:
    def __init__(self, batch_size, num_classes, vocab_size, sentence_len, embed_size, 
                 learning_rate, decay_steps, decay_rate, is_training):
        #1.定义超参数
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.sentence_len = sentence_len
        self.embed_size = embed_size
        self.hidden_size = embed_size #lstm层的维度
        self.learning_rate = learning_rate
        self.is_training = is_training
        self.initializer = tf.random_normal_initializer(stddev=0.1)
        
        #epoch信息
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.epoch_step = tf.Variable(0, trainable=False, name='epoch_step')
        self.epoch_increment = tf.assign(self.epoch_step, tf.add(self.epoch_step, tf.constant(1)))
        self.decay_steps, self.decay_rate = decay_steps, decay_rate
        
        #2.输入
        self.input_x = tf.placeholder(tf.int32, [None, sentence_len], 'input_x')
        self.input_y = tf.placeholder(tf.int32, [None], 'input_y') #单个标签
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        #3.初始化全连接层参数
        self.init_weight()
        
        #4.网络结构
        self.logits = self.inference() #[batch_size, num_classes]
        
        #5.损失函数
        self.loss_val = self.loss()
        
        #6.优化器
        self.train_op = self.train()
        
        #7.计算acc
        self.prediction = tf.argmax(self.logits, axis=1) #[batch_size]
        is_right = tf.equal(tf.cast(self.prediction, tf.int32), self.input_y)
        self.accuracy = tf.reduce_mean(tf.cast(is_right, tf.float32))
        
    def init_weight(self):
        self.Embedding = tf.get_variable('Embedding', [self.vocab_size, self.embed_size], dtype=tf.float32)
        self.W = tf.get_variable('W', [self.hidden_size * 2, self.num_classes], dtype=tf.float32) #双向LSTM，输出concat，所以此处为2倍
        self.b = tf.get_variable('b', [self.num_classes], dtype=tf.float32)
        
    def inference(self):
        # a.embedding
        self.sentence_embed = tf.nn.embedding_lookup(self.Embedding, self.input_x) #[batch_size, sentence_len, embed_size]
        
        # b.bidiretional lstm
        self.fw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #前向单元
        self.bw_cell = tf.contrib.rnn.BasicLSTMCell(self.hidden_size) #后向单元
#         if self.dropout_keep_prob is not None:
#             self.fw_cell = tf.contrib.rnn.DropoutWrapper(self.fw_cell, output_keep_prob=) 
                #input_keep_prob是对输入而言，output_keep_prb是对lstm各层而言
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(self.fw_cell, self.bw_cell, self.sentence_embed, dtype=tf.float32)
        #输入为 [batch_size, sentence_len, embed_size]，输出为大小为2的元组，每个元素为[batch_size, sentence_len, hidden_size]
        
        # c.concat
        fw_output = outputs[0][:,-1,:]
        bw_output = outputs[1][:,-1,:] #[batch_size, 1, hidden_size]
        final_output = tf.concat([fw_output, bw_output], axis=1) #[batch_size, 1, hidden_size*2]
        final_output = tf.reshape(final_output, [-1, self.hidden_size*2]) #[batch_size, hidden_size * 2]
        
        # d.full_connection
        logits = tf.matmul(final_output, self.W) + self.b #[batch_size, num_classes]
        return logits
    
    def loss(self, l2_lambda=0.0001):
        loss1 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.logits)
        #先将label转化为one-hot形式，再对logits计算softmax，最后计算交叉熵
        loss1 = tf.reduce_mean(loss1)
        loss2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()]) * l2_lambda
        return loss1 + loss2
    
    def train(self):
        learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps, self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, self.global_step, learning_rate, optimizer='Adam')
        return train_op
    

### 测试

In [3]:
def test():
    num_classes=19
    learning_rate=0.01
    batch_size=15
    decay_step=1000
    decay_rate=0.9
    sequence_length=5
    vocab_size=10000
    embed_size=100
    is_training=True
    dropout_keep_prob=0.5
    
    model = TextRNN(batch_size, num_classes, vocab_size, sequence_length, embed_size, 
                     learning_rate, decay_step, decay_rate, True)
    print(tf.trainable_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        input_x = np.random.randint(0,100,size=(batch_size, sequence_length),dtype=np.int32)
        input_y = np.random.randint(0, 19,size=(batch_size), dtype=np.int32)
        for i in range(20):
            #input_x = np.zeros((batch_size, sequence_length), dtype=np.int32)
            #input_y = np.array([1,0,1,1,1,2,1,1], dtype=np.int32)
            loss, acc, predict, _ = sess.run([model.loss_val, model.accuracy, model.prediction, model.train_op],
                                            feed_dict={model.input_x: input_x, model.input_y: input_y,
                                                       model.dropout_keep_prob: dropout_keep_prob})
            print('loss:',loss, 'acc:', acc, 'label:', input_y, 'predict:', predict)

In [4]:
tf.reset_default_graph()
test()

[<tf.Variable 'Embedding:0' shape=(10000, 100) dtype=float32_ref>, <tf.Variable 'W:0' shape=(200, 19) dtype=float32_ref>, <tf.Variable 'b:0' shape=(19,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/fw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/kernel:0' shape=(200, 400) dtype=float32_ref>, <tf.Variable 'bidirectional_rnn/bw/basic_lstm_cell/bias:0' shape=(400,) dtype=float32_ref>]
loss: 3.020527 acc: 0.0 label: [ 7 13 10  0  3  3 10 12  6 12  9 11 14 14  8] predict: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
loss: 2.9279802 acc: 0.2 label: [ 7 13 10  0  3  3 10 12  6 12  9 11 14 14  8] predict: [ 2 13 10  2  2  2 10  2  2  2  2  2  2  2  2]
loss: 2.7922528 acc: 0.46666667 label: [ 7 13 10  0  3  3 10 12  6 12  9 11 14 14  8] predict: [10 13 10 10 10 10 10 10 10 13 13 11 14 14  8]
loss: 2.5618093 acc: 0.46666667 label: [ 7 13 10  0

### 训练

In [5]:
import sys
import tensorflow as tf
import numpy as np
# from tflearn.data_utils import to_categorical, pad_sequences
import os
import pickle
import h5py

In [6]:
#定义超参数
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('batch_size', 512, 'batch_size')
tf.app.flags.DEFINE_integer('num_classes', 1999, 'num_classes')
tf.app.flags.DEFINE_integer('sentence_len', 100, 'length of each sentence')
tf.app.flags.DEFINE_integer('embed_size', 100, 'embedding size')
tf.app.flags.DEFINE_float('learning_rate', 0.01, '')
tf.app.flags.DEFINE_float('decay_rate', 0.8, '')
tf.app.flags.DEFINE_integer('decay_steps', 3000, 'number of steps before decay learning rate')
tf.app.flags.DEFINE_bool('is_training', True, '')

tf.app.flags.DEFINE_integer('num_epoch', 20, 'number of epoch')

tf.app.flags.DEFINE_string("ckpt_dir","testrnn_singlelabel_checkpoint_noembed/","checkpoint location for the model")
tf.app.flags.DEFINE_string("cache_path","textrnn_singlelabel_checkpoint_noembed/data_cache.pik","data chche for the model")

In [7]:
import time
def log(str):
    t = time.localtime()
    print("[%4d/%02d/%02d %02d:%02d:%02d]"%(t.tm_year, t.tm_mon, t.tm_mday, t.tm_hour, t.tm_min, t.tm_sec), end=' ')
    print(str)

In [14]:
def main(_):
    #1.加载数据
    base_path = '../zhihu_data/'
    cache_file_h5py = base_path + 'data.h5'
    cache_file_pickle = base_path + 'vocab_label.pik'
    word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y,embedding_final = load_data(cache_file_h5py, cache_file_pickle)
    vocab_size = len(word2index)
    
    print("train_X.shape:", np.array(train_X).shape)
    print("train_y.shape:", np.array(train_y).shape)
    print("test_X.shape:", np.array(test_X).shape)  # 每个list代表一句话
    print("test_y.shape:", np.array(test_y).shape)  
    print("test_X[0]:", test_X[0])  
    print("test_y[0]:", test_y[0]) 
    
    #2.创建session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        model = TextRNN(FLAGS.batch_size, FLAGS.num_classes, vocab_size, FLAGS.sentence_len,FLAGS.embed_size, 
                        FLAGS.learning_rate, FLAGS.decay_steps, FLAGS.decay_rate, FLAGS.is_training)
        saver = tf.train.Saver()
        batch_size = FLAGS.batch_size
        CONTINUE_TRAIN = False
        if os.path.exists(FLAGS.ckpt_dir + 'checkpoint'):
            log("restore from checkpoint")
            saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
            if CONTINUE_TRAIN: log("continue training...")
        else:
            log('init variables')
            sess.run(tf.global_variables_initializer())
#             #是否使用embedding
#             print('assign pre-trained embedding')
#             embedding_assign = tf.assign(model.Embedding, tf.constant(np.array(embedding_final))) #为model.Embedding赋值
#             sess.run(embedding_assign)
        if not os.path.exists(FLAGS.ckpt_dir + 'checkpoint') or CONTINUE_TRAIN:
            num_of_data = len(train_y)
            for _ in range(FLAGS.num_epoch):
                epoch = sess.run(model.epoch_step)
                loss, acc, counter = 0., 0., 0.
                for start, end in zip(range(0, num_of_data, batch_size), range(batch_size, num_of_data, batch_size)):
                    loss_tmp, acc_tmp, _ = sess.run([model.loss_val, model.accuracy, model.train_op], 
                                                    feed_dict={model.input_x: train_X[start:end,:100], model.input_y: train_y[start:end],
                                                               model.dropout_keep_prob: 1})
                    loss, acc, counter = loss + loss_tmp, acc + acc_tmp, counter + 1
                    if counter % 200 == 0:
                        log("Epoch %d\Batch %d\ Train Loss:%.3f\ Train Accuracy:%.3f"%(epoch, counter, loss/float(counter), acc/float(counter)))

                print('run model on validation data...')
                loss_valid, acc_valid = do_eval(sess, model, vaild_X, valid_y, batch_size)
                log("Epoch %d\ Validation Loss:%.3f/ Validation Accuracy:%.3f"%(epoch, loss_valid, acc_valid))
                #save the checkpoint
                save_path = FLAGS.ckpt_dir + 'model.ckpt'
                saver.save(sess, save_path, global_step=model.epoch_step)
                sess.run(model.epoch_increment)
            loss_valid, acc_valid = do_eval(sess, model, vaild_X, valid_y, batch_size)
            log("Validation Loss:%.3f\ Validation Accuracy:%.3f"%(loss_valid, acc_valid))
        loss_valid, acc_valid = do_eval(sess, model, vaild_X, valid_y, batch_size)
        log("Validation Loss:%.3f\ Validation Accuracy:%.3f"%(loss_valid, acc_valid))
            
    

In [15]:
def load_data(h5_file_path, pik_file_path):
    if not os.path.exists(h5_file_path) or not os.path.exists(pik_file_path):
        raise RuntimeError('No such file!!')

    print('cache files exist, going to load in...')
    print('loading h5_file...')
    h5_file = h5py.File(h5_file_path, 'r')
    print('h5_file.keys:', h5_file.keys())
    train_X, train_y = h5_file['train_X'], h5_file['train_Y']
    vaild_X, valid_y = h5_file['vaild_X'], h5_file['valid_Y']
    test_X,  test_y  = h5_file['test_X'],  h5_file['test_Y']
    embedding_final = h5_file['embedding']

    print('loading pickle file')
    word2index, label2index = None, None
    with open(pik_file_path, 'rb') as pkl:
        word2index,label2index = pickle.load(pkl)
    print('cache files load successful!')
    return word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y, embedding_final

def do_eval(sess, model, test_X, test_y, batch_size):
#     test_X, test_y = test_X[:5000], test_y[:5000]
    num_of_data = len(test_y)
    loss, acc, counter = 0.0, 0.0, 0
    for start, end in zip(range(0, num_of_data, batch_size), range(batch_size, num_of_data, batch_size)):
        l,a = sess.run([model.loss_val, model.accuracy], 
                        feed_dict={model.input_x: test_X[start:end,:100], model.input_y: test_y[start:end], model.dropout_keep_prob:1.0})
        loss, acc, counter = loss+l, acc+a, counter+1
    return loss/float(counter), acc/float(counter)

In [10]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X.shape: (2959966, 200)
train_y.shape: (2959966,)
test_X.shape: (20000, 200)
test_y.shape: (20000,)
test_X[0]: [ 579  343 1173 1843    5  583  292 1173 1843    5 1180 1299  989   10
    2   68  153  168  531  109  260  217  277   81   59   81  116  514
    6  221  253  224  154  718  553    4  806  538  732  264   74    6
  221  224  154  326   11  167  136    4  257  145   37   74  175  214
   11   57  110  221    6  364   89   20 4050 2344    4  257   78    9
  991  326  221   89  699  133   11  597  679 1957  824  884  871 1957
  824    4  178   87   87   78  196   52  552   69   47   20   12   37
 1371   89    6  755  779   81  667  597    4  586  878    6   35   93
    7  719  285  937   35  162   13   11    7 1371   89   35    4  201
   68   81   97 1533   81  667  597    9  991  326   35  343  704   16
    5   99 

[2019/03/23 10:56:58] Epoch 11\Batch 5000\ Train Loss:4.170\ Train Accuracy:0.289
[2019/03/23 10:58:45] Epoch 11\Batch 5200\ Train Loss:4.170\ Train Accuracy:0.289
[2019/03/23 11:00:33] Epoch 11\Batch 5400\ Train Loss:4.171\ Train Accuracy:0.289
[2019/03/23 11:02:20] Epoch 11\Batch 5600\ Train Loss:4.170\ Train Accuracy:0.289
run model on validation data...
[2019/03/23 11:03:58] Epoch 11\ Validation Loss:4.274/ Validation Accuracy:0.274
[2019/03/23 11:05:46] Epoch 12\Batch 200\ Train Loss:4.178\ Train Accuracy:0.289
[2019/03/23 11:07:34] Epoch 12\Batch 400\ Train Loss:4.175\ Train Accuracy:0.290
[2019/03/23 11:09:18] Epoch 12\Batch 600\ Train Loss:4.169\ Train Accuracy:0.290
[2019/03/23 11:10:56] Epoch 12\Batch 800\ Train Loss:4.168\ Train Accuracy:0.289
[2019/03/23 11:12:43] Epoch 12\Batch 1000\ Train Loss:4.169\ Train Accuracy:0.289
[2019/03/23 11:14:29] Epoch 12\Batch 1200\ Train Loss:4.168\ Train Accuracy:0.289
[2019/03/23 11:16:15] Epoch 12\Batch 1400\ Train Loss:4.167\ Train Accu

[2019/03/23 13:52:48] Epoch 15\Batch 1600\ Train Loss:4.156\ Train Accuracy:0.290
[2019/03/23 13:54:39] Epoch 15\Batch 1800\ Train Loss:4.156\ Train Accuracy:0.290
[2019/03/23 13:56:15] Epoch 15\Batch 2000\ Train Loss:4.156\ Train Accuracy:0.290
[2019/03/23 13:58:07] Epoch 15\Batch 2200\ Train Loss:4.154\ Train Accuracy:0.291
[2019/03/23 13:59:59] Epoch 15\Batch 2400\ Train Loss:4.154\ Train Accuracy:0.291
[2019/03/23 14:01:51] Epoch 15\Batch 2600\ Train Loss:4.154\ Train Accuracy:0.290
[2019/03/23 14:03:42] Epoch 15\Batch 2800\ Train Loss:4.154\ Train Accuracy:0.290
[2019/03/23 14:05:33] Epoch 15\Batch 3000\ Train Loss:4.155\ Train Accuracy:0.290
[2019/03/23 14:07:27] Epoch 15\Batch 3200\ Train Loss:4.155\ Train Accuracy:0.291
[2019/03/23 14:09:19] Epoch 15\Batch 3400\ Train Loss:4.156\ Train Accuracy:0.291
[2019/03/23 14:11:07] Epoch 15\Batch 3600\ Train Loss:4.155\ Train Accuracy:0.291
[2019/03/23 14:12:54] Epoch 15\Batch 3800\ Train Loss:4.155\ Train Accuracy:0.291
[2019/03/23 14:1

[2019/03/23 16:57:39] Epoch 18\Batch 4000\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 16:59:25] Epoch 18\Batch 4200\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 17:01:13] Epoch 18\Batch 4400\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 17:03:01] Epoch 18\Batch 4600\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 17:04:47] Epoch 18\Batch 4800\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 17:06:33] Epoch 18\Batch 5000\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 17:08:18] Epoch 18\Batch 5200\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 17:11:50] Epoch 18\Batch 5600\ Train Loss:4.151\ Train Accuracy:0.291
run model on validation data...
[2019/03/23 17:13:29] Epoch 18\ Validation Loss:4.268/ Validation Accuracy:0.273
[2019/03/23 17:15:14] Epoch 19\Batch 200\ Train Loss:4.161\ Train Accuracy:0.291
[2019/03/23 17:17:03] Epoch 19\Batch 400\ Train Loss:4.159\ Train Accuracy:0.292
[2019/03/23 17:18:52] Epoch 19\Batch 600\ Train Loss:4.154\ Train Acc

[2019/03/23 19:57:52] Epoch 22\Batch 800\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 19:59:44] Epoch 22\Batch 1000\ Train Loss:4.153\ Train Accuracy:0.291
[2019/03/23 20:01:36] Epoch 22\Batch 1200\ Train Loss:4.153\ Train Accuracy:0.291
[2019/03/23 20:03:28] Epoch 22\Batch 1400\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 20:05:24] Epoch 22\Batch 1600\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 20:07:19] Epoch 22\Batch 1800\ Train Loss:4.152\ Train Accuracy:0.291
[2019/03/23 20:09:14] Epoch 22\Batch 2000\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 20:11:09] Epoch 22\Batch 2200\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 20:13:05] Epoch 22\Batch 2400\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 20:14:48] Epoch 22\Batch 2600\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 20:16:45] Epoch 22\Batch 2800\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 20:18:40] Epoch 22\Batch 3000\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 20:20

[2019/03/23 23:01:33] Epoch 25\Batch 3200\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 23:03:20] Epoch 25\Batch 3400\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 23:05:02] Epoch 25\Batch 3600\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 23:06:40] Epoch 25\Batch 3800\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:08:27] Epoch 25\Batch 4000\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:10:13] Epoch 25\Batch 4200\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 23:11:59] Epoch 25\Batch 4400\ Train Loss:4.151\ Train Accuracy:0.291
[2019/03/23 23:13:46] Epoch 25\Batch 4600\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:15:33] Epoch 25\Batch 4800\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:17:20] Epoch 25\Batch 5000\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:19:06] Epoch 25\Batch 5200\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:20:54] Epoch 25\Batch 5400\ Train Loss:4.150\ Train Accuracy:0.291
[2019/03/23 23:2

[2019/03/24 01:56:10] Epoch 28\Batch 5600\ Train Loss:4.150\ Train Accuracy:0.291
run model on validation data...
[2019/03/24 01:57:47] Epoch 28\ Validation Loss:4.268/ Validation Accuracy:0.273
[2019/03/24 01:57:49] Validation Loss:4.268\ Validation Accuracy:0.273


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X.shape: (2959966, 200)
train_y.shape: (2959966,)
test_X.shape: (20000, 200)
test_y.shape: (20000,)
test_X[0]: [ 579  343 1173 1843    5  583  292 1173 1843    5 1180 1299  989   10
    2   68  153  168  531  109  260  217  277   81   59   81  116  514
    6  221  253  224  154  718  553    4  806  538  732  264   74    6
  221  224  154  326   11  167  136    4  257  145   37   74  175  214
   11   57  110  221    6  364   89   20 4050 2344    4  257   78    9
  991  326  221   89  699  133   11  597  679 1957  824  884  871 1957
  824    4  178   87   87   78  196   52  552   69   47   20   12   37
 1371   89    6  755  779   81  667  597    4  586  878    6   35   93
    7  719  285  937   35  162   13   11    7 1371   89   35    4  201
   68   81   97 1533   81  667  597    9  991  326   35  343  704   16
    5   99 

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
